In [25]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf

from Functions.conn import db_upload
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import load_model
from sklearn.preprocessing import minmax_scale
print(tf.__version__)


2.0.0-alpha0


In [52]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="a3d"
)

def db_upload(sql, val):
  mycursor = mydb.cursor()
  mycursor.execute(sql, val)
  mydb.commit()
  print(mycursor.rowcount, "record inserted.")




#function to create trainingArray that groups data into lists of 'groupSize' date rows (atm 288) * all indices' data (8)
#creating a master list of 780 rows of 2304 columns, essentially 780 moving windows of all data
def groupData( data, groupSize ):
	trainingArray = []
	for i in range( data.size ):
		if i >= groupSize and i <= (data.shape[0]-days_indicator) :
			trainingListSingleEntry = []
			for ii in range( groupSize ):
				trainingListSingleEntry = np.concatenate( ( trainingListSingleEntry, data[ i - ii -1] ), axis=None )
			trainingArray.append( trainingListSingleEntry )
	return trainingArray


def groupLabels( data, groupSize ):
	trainingArray = []
	for i in range( data.size):
		if i >= (groupSize+days_indicator-1):
			trainingListSingleEntry = data[i]
			trainingArray.append( trainingListSingleEntry )
	return trainingArray

def doTraining():
	trainingData = index_data[ subset ].values
	trainingData = minmax_scale( trainingData )
	trainingData = groupData( trainingData, groupSize = groupSize )
	trainingData = np.array( trainingData )
# 	print("training data 1", trainingData[0])
	print("trainingData groupData shape expecting 752 x 2304", trainingData.shape)

#trainingLabels being target, output dataset
	trainingLabels = index_data[ 'AXJOA' ].values
	trainingLabels = minmax_scale( trainingLabels )
# 	print("training label size is ")
# 	print(trainingLabels.size)
	trainingLabels = groupLabels( trainingLabels, groupSize = groupSize )
	trainingLabels = np.array( trainingLabels )
	print("training label row 1",trainingLabels[0])
	print("training label shape expecting 753 * 1 ", trainingLabels.shape)

	model = Sequential()
	print("input layer set to:", len( trainingData[0] ))
	model.add( Dense( 32, input_dim = len( trainingData[0] ), kernel_initializer = 'normal', activation = 'relu' ) ) # going into a 32-unit layer
	model.add( Dense( 32, kernel_initializer = 'normal', activation = 'relu' ) ) # Another hidden layer of 16 units
	model.add( Dense( 32, kernel_initializer = 'normal', activation = 'relu' ) ) # Another hidden layer of 16 units
	model.add( Dense( 1 ) ) # Output layer with a binary classification ( Democrat or Republican )
	model.compile( loss = 'mse', optimizer = 'rmsprop', metrics = [ 'mae' ] ) # Compile model
	model.fit( trainingData, trainingLabels, epochs = 1, batch_size = 50, verbose = verbosity ) # Train model
	scores = model.evaluate( trainingData, trainingLabels, verbose = verbosity ) # Grade the model
	print( "%s: %.2f%%" % ( model.metrics_names[1], scores[1]*100 ) )
	model.save( 'BHMarket_Model.h5' ) # Save the model
# 	plot_model(model, to_file='modelplotbh.png')
    
def doPrediction():
	trainingData = index_data[ subset ].values
	print("trainingData groupData shape expecting unsure", trainingData.shape)
	ASAmin =  float((trainingData.min(axis=0))[0])
	print("min in trainingData is: ", ASAmin)
	ASAmax =  float((trainingData.max(axis=0))[0])
	print("max in trainingData is: ", ASAmax)
	ASArange =  ASAmax-ASAmin
	print("trainingData groupData shape expecting unsure", trainingData.shape)
	trainingData = minmax_scale( trainingData )
	trainingData = groupData( trainingData, groupSize = groupSize )
	inputData = trainingData[-1]
	print( "input data is: ", inputData )

	trainingLabels = index_data[ 'Date' ].values
	trainingLabels = groupLabels( trainingLabels, groupSize = groupSize )
# 	date = trainingLabels[-1]
# 	print( "training data shape is", inputData.shape )

	loaded_model = load_model( 'BHMarket_Model.h5' )

	sql = "INSERT INTO daily_ai_indicators (metric, value) VALUES (%s, %s)"
	# loaded_model = load_model( 'LSMarket_Model.h5' )
	# evaluate loaded model on test data
	loaded_model.compile( loss = 'mse', optimizer = 'rmsprop', metrics = [ 'mae' ] )
	# Predict things...
	print( "input data shape is: ", inputData.shape )
	prediction_1 = loaded_model.predict( inputData.reshape( (1, 576) ), batch_size = None, verbose = verbosity, steps = None )
	print("prediction scaled is:", prediction_1)    
	prediction_1=prediction_1*ASArange+ASAmin
	print("prediction is:", prediction_1)
	x123=prediction_1.tolist()
	final_int=(prediction_1.tolist())[0][0]
	metric='t_'+str(forecast_array[i])
	val = (metric, final_int)
	db_upload(sql,val)
    
#     print('t_'+str(i))
#     i=i+1 
# forecast_array=(1,7,30,90) # being t+1, 7, 30, 90 days array

forecast_array=(1,7,30,90)
array_length=len(forecast_array)
i=0

while (i<array_length):

    verbosity = 1
    groupSize = 288
    days_indicator=forecast_array[i]  #set equal to t+X days indicator
    index_names = [ 'Date', 'AXJOA', 'SPXTR']
    subset = [ 'AXJOA', 'SPXTR' ]
    index_data = pd.read_csv( 'spxt_axjoa.csv', names = index_names ) # read file, insert header row
    index_data.dropna( inplace = True ) 
    print("index_data shape is ")
    print(index_data.shape)
    index_data.to_csv('out.csv', index=False)
    ASAMin=0
    ASAMax=0
    ASArange=0
    doTraining()
    print("prediction is: ")
    doPrediction()
    i=i+1


index_data shape is 
(4598, 3)
trainingData groupData shape expecting 752 x 2304 (4310, 576)
training label row 1 0.8127757961653643
training label shape expecting 753 * 1  (4310,)
input layer set to: 576
4310/4310 [==============================] - 0s 41us/sample - loss: 0.0019 - mae: 0.0340
mae: 3.40%
prediction is: 
trainingData groupData shape expecting unsure (4598, 2)
min in trainingData is:  14050.6
max in trainingData is:  77845.45
trainingData groupData shape expecting unsure (4598, 2)
input data is:  [0.01202918 0.0613838  0.01394313 0.05736234 0.00994594 0.04970966
 0.00679052 0.04516501 0.00559606 0.05429747 0.00855398 0.05644029
 0.01078614 0.05524888 0.01153228 0.05805993 0.01531158 0.06348173
 0.01912537 0.06290156 0.02236074 0.0660027  0.02135282 0.07113787
 0.02172119 0.07049554 0.02554752 0.07097902 0.0225269  0.06880857
 0.02355363 0.06744276 0.02231528 0.07333767 0.02717461 0.07734532
 0.02918104 0.07594325 0.0271699  0.0755392  0.0280775  0.07737467
 0.03017642 0.0

input data shape is:  (576,)
1/1 [==============================] - 0s 11ms/sample
prediction scaled is: [[0.07301249]]
prediction is: [[18708.42]]
1 record inserted.
index_data shape is 
(4598, 3)
trainingData groupData shape expecting 752 x 2304 (4281, 576)
training label row 1 0.7322344985527829
training label shape expecting 753 * 1  (4281,)
input layer set to: 576
4281/4281 [==============================] - 0s 38us/sample - loss: 0.0093 - mae: 0.0786
mae: 7.86%
prediction is: 
trainingData groupData shape expecting unsure (4598, 2)
min in trainingData is:  14050.6
max in trainingData is:  77845.45
trainingData groupData shape expecting unsure (4598, 2)
input data is:  [0.03168438 0.07275405 0.0314226  0.06827329 0.0311812  0.06829747
 0.03135833 0.07423382 0.03068743 0.07499184 0.0295933  0.07879746
 0.03418771 0.08268942 0.0316734  0.08203846 0.03297288 0.08271705
 0.03426766 0.08315217 0.03597156 0.08267043 0.03491034 0.08775553
 0.03840279 0.08833742 0.03901412 0.08858779 0.03

input data shape is:  (576,)
1/1 [==============================] - 0s 12ms/sample
prediction scaled is: [[0.0678926]]
prediction is: [[18381.797]]
1 record inserted.


In [ ]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="a3d"
)

def db_upload(sql, val):
  mycursor = mydb.cursor()
  mycursor.execute(sql, val)
  mydb.commit()
  print(mycursor.rowcount, "record inserted.")


sql = "INSERT INTO daily_ai_indicators (metric, value) VALUES (%s, %s)"
val = ("delta_strategy", 123477)
db_upload(sql,val)

In [47]:
forecast_array=(1,7,30,90)


array_length=(len(forecast_array))
i=0
while (i<array_length):
    print('t_'+str(i))
    i=i+1



t_0
t_1
t_2
t_3
